In [1]:
import sys
sys.path.insert(1, '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib')
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math 
import timeit

from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm

In [2]:
true_effect = 30
data = dowhy.datasets.linear_dataset(beta=true_effect,
        num_common_causes=5,
        num_instruments = 0,
        num_samples=10000,
        treatment_is_binary=True)
df_expr = data["df"].copy()
df_expr['v0'] = [1 if x == True else 0 for x in df_expr['v0']]

D0, D1 = train_test_split(df_expr, train_size = 0.9, random_state = 1)
D0 = D0.reset_index().drop(columns = ['index'])
D1 = D1.reset_index().drop(columns = ['index'])

In [3]:
treatment = data['treatment_name'][0]
covariate = data['common_causes_names']
outcome = data['outcome_name']
n_splits = 5
delta = np.linspace(0.5, 5, n_splits)

# effect = ipse.proposed_estimation(df_expr, treatment, covariate, outcome, delta, n_splits)

In [4]:
# ipse.sample_estimator(D0, D1, treatment, covariate, outcome, delta[0])

In [5]:
# effect

In [6]:
# np.mean(effect)

In [7]:
def evaluation(true_effect):
    data = dowhy.datasets.linear_dataset(beta=true_effect,
            num_common_causes=7,
            num_instruments = 0,
            num_samples=40000,
            treatment_is_binary=True)
    df_expr = data["df"].copy()
    df_expr['v0'] = [1 if x == True else 0 for x in df_expr['v0']]

    D0, D1 = train_test_split(df_expr, train_size = 0.9, random_state = 1)
    D0 = D0.reset_index().drop(columns = ['index'])
    D1 = D1.reset_index().drop(columns = ['index'])
    treatment = data['treatment_name'][0]
    covariate = data['common_causes_names']
    outcome = data['outcome_name']
    n_splits = 5
    delta = np.linspace(0.5, 5, n_splits)
    e = ipse.proposed_estimation(df_expr, treatment, covariate, outcome, delta, n_splits)
    return np.mean(e)

In [8]:
# for i in np.linspace(10,50,25):
#     print("True effect {}".format(i))
#     effect = evaluation(i)
#     print("Causal effect {}".format(effect))
#     print("MAE {}".format(abs(i - abs(effect))))
#     print("\n")
    

In [9]:
def evaluation_inc(true_effect, data):
#     data = dowhy.datasets.linear_dataset(beta=true_effect,
#             num_common_causes=7,
#             num_instruments = 0,
#             num_samples=40000,
#             treatment_is_binary=True)
    df_expr = data["df"].copy()
    df_expr['v0'] = [1 if x == True else 0 for x in df_expr['v0']]

    rfc = RandomForestClassifier(max_depth=50, random_state=0)
    rfc.fit(df_expr[data['common_causes_names']], df_expr[data['treatment_name'][0]])

    features = data['common_causes_names'] + data['treatment_name']
    rf = RandomForestRegressor(n_estimators = 100, random_state=0)
    rf.fit(df_expr[features], df_expr['y'])
    
    treatment = data['treatment_name'][0]
    covariate = data['common_causes_names']
    outcome = data['outcome_name']
    e = ipse.incremenal_ps_score_estimator(df_expr, treatment, covariate, rfc, rf, delta = 2)
    
    att, atc, est= pse.propensity_score_estimator_k_nearest_neighbor(df_expr, 
                                                                     treatment, 
                                                                     outcome, 
                                                                     'ps_1')
    return np.mean(e), est 



In [21]:
for i in np.linspace(10,50,25):
    print("True effect {}".format(i))
    data = dowhy.datasets.linear_dataset(beta=10,
                num_common_causes=7,
                num_instruments = 0,
                num_samples=40000,
                treatment_is_binary=True)    
   
    effect, est = evaluation_inc(10, data)
    print("Causal effect of incremental {} and {}".format(effect, est))
    print("MAE of incremental and nearest {} and {}".format(abs(10 - abs(effect)),
                                                     abs(10 - abs(est))
                                                    ))
    print("\n")
    
    if abs(10 - abs(effect)) > 5:
        break

True effect 10.0
Causal effect of incremental 3.206856135761271 and 13.419461486152649
MAE of incremental and nearest 6.793143864238729 and 3.419461486152649




In [ ]:
treatment = data['treatment_name'][0]
covariate = data['common_causes_names']
outcome = data['outcome_name']
    
features = covariate.copy()

features.append(treatment)

df_expr = data["df"].copy()
df_expr['v0'] = [1 if x == True else 0 for x in df_expr['v0']]

rfc = RandomForestClassifier(max_depth=50, random_state=0)
rfc.fit(df_expr[data['common_causes_names']], df_expr[data['treatment_name'][0]])

features = data['common_causes_names'] + data['treatment_name']
rf = RandomForestRegressor(n_estimators = 100, random_state=0)
rf.fit(df_expr[features], df_expr['y'])
    
    

model_t = 

## Compute propensity_score
df['ps_1'] = model_t.predict_proba(df[covariate])[:,1]
df['ps_0'] = model_t.predict_proba(df[covariate])[:,0]

## Compute incremental propensity score
df['incre_ps_1'] = (delta * df['ps_1']) / (delta * df['ps_1'] + df['ps_0'])
df['incre_ps_0'] = 1 - df['incre_ps_1']

## Compute counterfactual outcome with no treatment
df_pos = df.copy()
df_pos['v0'] = 0
df['treated_cf_outcome'] = model_y.predict(df_pos[features])

## Compute counterfactual outcome with treatment
df_control = df.copy()
df_control['v0'] = 1
df['control_cf_outcome'] = model_y.predict(df_control[features])

df['incre_effect'] = df['incre_ps_1']*df['treated_cf_outcome'] + df['incre_ps_0']*df['control_cf_outcome']

In [23]:
import regex as re
filename = "abstract{aaaaa} (aaaaa)"
re.sub(r'abstract\{[^)]*\}', '', filename)


' (aaaaa)'

In [69]:
text = '''@article{lipton_mythos_2017,
	title = {The {Mythos} of {Model} {Interpretability}},
	url = {http://arxiv.org/abs/1606.03490},
	abstract{Supervised machine learning models boast remarkable predictive capabilities. But can you trust your model? Will it work in deployment? What else can it tell you about the world? We want models to be not only good, but interpretable. And yet the task of interpretation appears underspecified. Papers provide diverse and sometimes non-overlapping motivations for interpretability, and offer myriad notions of what attributes render models interpretable. Despite this ambiguity, many papers proclaim interpretability axiomatically, absent further explanation. In this paper, we seek to refine the discourse on interpretability. First, we examine the motivations underlying interest in interpretability, finding them to be diverse and occasionally discordant. Then, we address model properties and techniques thought to confer interpretability, identifying transparency to humans and post-hoc explanations as competing notions. Throughout, we discuss the feasibility and desirability of different notions, and question the oft-made assertions that linear models are interpretable and that deep neural networks are not.},
	urldate = {2020-03-08},
	journal = {arXiv:1606.03490 [cs, stat]},
	author = {Lipton, Zachary C.},
	month = mar,
	year = {2017},
	note = {arXiv: 1606.03490},
	keywords = {Computer Science - Machine Learning, Statistics - Machine Learning, Computer Science - Artificial Intelligence, Computer Science - Computer Vision and Pattern Recognition, Computer Science - Neural and Evolutionary Computing},
	annote = {Comment: presented at 2016 ICML Workshop on Human Interpretability in Machine Learning (WHI 2016), New York, NY},
	file = {3236386.3241340.pdf:C\:\\Users\\13762012\\Zotero\\storage\\3BK4MMPE\\3236386.3241340.pdf:application/pdf;arXiv.org Snapshot:C\:\\Users\\13762012\\Zotero\\storage\\DUXP935F\\1606.html:text/html;arXiv Fulltext PDF:C\:\\Users\\13762012\\Zotero\\storage\\ILE5E4PW\\Lipton - 2017 - The Mythos of Model Interpretability.pdf:application/pdf}
}'''

In [70]:
# text = '''abstract{aaaa}'''
# text = """abstract{Supervised mac},assdsda urldate = {2020-03-08}, aaaaaaaaaaaaaaaaaaa, asdsd"""
# text = """
# abstract{aaaa} url = {}
# """

re.sub(r'abstract\{[^}]*\}', '', text)


'@article{lipton_mythos_2017,\n\ttitle = {The {Mythos} of {Model} {Interpretability}},\n\turl = {http://arxiv.org/abs/1606.03490},\n\t,\n\turldate = {2020-03-08},\n\tjournal = {arXiv:1606.03490 [cs, stat]},\n\tauthor = {Lipton, Zachary C.},\n\tmonth = mar,\n\tyear = {2017},\n\tnote = {arXiv: 1606.03490},\n\tkeywords = {Computer Science - Machine Learning, Statistics - Machine Learning, Computer Science - Artificial Intelligence, Computer Science - Computer Vision and Pattern Recognition, Computer Science - Neural and Evolutionary Computing},\n\tannote = {Comment: presented at 2016 ICML Workshop on Human Interpretability in Machine Learning (WHI 2016), New York, NY},\n\tfile = {3236386.3241340.pdf:C\\:\\Users\\13762012\\Zotero\\storage\\3BK4MMPE\\3236386.3241340.pdf:application/pdf;arXiv.org Snapshot:C\\:\\Users\\13762012\\Zotero\\storage\\DUXP935F\\1606.html:text/html;arXiv Fulltext PDF:C\\:\\Users\\13762012\\Zotero\\storage\\ILE5E4PW\\Lipton - 2017 - The Mythos of Model Interpretabili